# EDCL Tier‑A reproduction (CLASS v3.4 + Cobaya v3.6)

This notebook is the **referee‑oriented** workflow for reproducing the manuscript’s Tier‑A cosmology results.

What this notebook does:
1. Clones **CLASS v3.4**.
2. Applies the manuscript’s `class_edcl.patch` (you must supply the exact patch file).
3. Builds `classy`.
4. Installs **Cobaya v3.6**.
5. Renders YAMLs (no guessed paths).
6. Runs **smoke tests** (no external datasets).
7. (Optional) Runs Cobaya chains after installing likelihood datasets.

Important:
- Planck 2018 likelihoods require external packages/data; installation can be nontrivial on Colab.
- Start with the “late‑only” smoke chain first.


In [ ]:
# If you uploaded a zip (e.g., TP-EDCL-repro-pack-v4.zip) to Colab, set ZIP_NAME accordingly.
ZIP_NAME = None  # e.g. "TP-EDCL-repro-pack-v4.zip"

import os, zipfile
if ZIP_NAME:
    with zipfile.ZipFile(ZIP_NAME, "r") as z:
        z.extractall(".")
    print("Extracted zip.")
else:
    print("If you are using a git clone instead of a zip, you can ignore this cell.")


In [ ]:
# Locate the pack directory
import os, pathlib
PACK_DIR = "TP-EDCL-simulation-pack"
assert os.path.isdir(PACK_DIR), f"Expected {PACK_DIR} in current directory."
print("Found pack at", os.path.abspath(PACK_DIR))


In [ ]:
# (1) Clone CLASS v3.4
!rm -rf class_public
!git clone https://github.com/lesgourg/class_public.git
%cd class_public
!git checkout v3.4
%cd ..


In [ ]:
# (2) Apply EDCL patch (you must provide the exact patch file)
import shutil, os, pathlib

patch_src = os.path.join(PACK_DIR, "cosmology", "patches", "class_edcl.patch")
patch_dst = os.path.join("class_public", "class_edcl.patch")
print("Patch source:", patch_src)
print("Patch destination:", patch_dst)

# Safety: stop if the patch is still the placeholder.
with open(patch_src, "r", encoding="utf-8") as f:
    content = f.read()
if "PLACEHOLDER" in content:
    raise RuntimeError("The patch file is still a placeholder. Replace cosmology/patches/class_edcl.patch with the real patch from the manuscript authors/repo.")

shutil.copyfile(patch_src, patch_dst)

%cd class_public
!patch -p1 < class_edcl.patch
%cd ..


In [ ]:
# (3) Build CLASS python wrapper (classy)
%cd class_public
!make clean
!make classy -j2
%cd ..


In [ ]:
# (4) Install Cobaya v3.6
!pip -q install cobaya==3.6


In [ ]:
# (5) Discover installed Cobaya component keys (no assumptions)
import os, sys, subprocess
subprocess.check_call([sys.executable, os.path.join(PACK_DIR, "cosmology", "scripts", "discover_cobaya_components.py")], cwd=PACK_DIR)
print("Wrote:", os.path.join(PACK_DIR, "cosmology", "paper_artifacts", "cobaya_components.txt"))


In [ ]:
# (6) Render YAMLs with the concrete CLASS path and an output chains/ directory
import os, sys, subprocess
class_path = os.path.abspath("class_public")
out_root = os.path.abspath(os.path.join(PACK_DIR, "chains"))
subprocess.check_call([sys.executable, os.path.join("cosmology", "scripts", "render_yamls.py"),
                       "--class-path", class_path,
                       "--out-root", out_root],
                      cwd=PACK_DIR)


In [ ]:
# (7) Smoke test: verify patched CLASS accepts EDCL parameters (no external data needed)
import os, sys, subprocess
class_path = os.path.abspath("class_public")
subprocess.check_call([sys.executable, os.path.join(PACK_DIR, "cosmology", "scripts", "smoke_test_classy_edcl.py"),
                       "--class-path", class_path])


In [ ]:
# (8) Background-only H(z) ratio plot from CLASS (no external data needed)
import os, sys, subprocess
class_path = os.path.abspath("class_public")
subprocess.check_call([sys.executable, os.path.join(PACK_DIR, "cosmology", "scripts", "make_fig_hubble_ratio_from_class.py"),
                       "--class-path", class_path,
                       "--alpha_R", "0.118",
                       "--log10_l0", "-20.91"])


## Optional: install likelihood data and run chains

If you want to attempt the full paper configuration on Colab:

1. Install likelihood packages/data via Cobaya:
   - One common method is to run `cobaya-install` on the YAML you plan to run.
   - Example (may take a while and may require additional system deps):
     - `cobaya-install cosmology/cobaya/edcl_cosmo_lateonly.yaml`
     - `cobaya-install cosmology/cobaya/edcl_cosmo_full.yaml`

2. Run the late-only smoke chain first:
   - `cobaya-run cosmology/cobaya/lcdm_lateonly.yaml`
   - `cobaya-run cosmology/cobaya/edcl_cosmo_lateonly.yaml`

3. Only then attempt the full Planck+DESI+PantheonPlus+SH0ES chain.

If the Planck likelihood installation fails (common on Colab), run the full chains on a local Linux box
with `clik` available, but keep this notebook for the no-data smoke tests.


In [ ]:
# OPTIONAL: attempt to install late-only likelihoods and run short chains
# Uncomment if you want to try in Colab.

# %cd TP-EDCL-simulation-pack
# !cobaya-install cosmology/cobaya/edcl_cosmo_lateonly.yaml
# !cobaya-run cosmology/cobaya/lcdm_lateonly.yaml
# !cobaya-run cosmology/cobaya/edcl_cosmo_lateonly.yaml
